<a href="https://colab.research.google.com/github/shahryaramd/flood-mapping/blob/master/flood_water_classification_ml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table class="ee-notebook-buttons" align="left"><td>
<a target="_blank"  href="https://github.com/shahryaramd/flood-mapping/blob/master/flood_water_classification_ml.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td></table>

Written by Shahryar Ahmad (skahmad@uw.edu)

*(modified from [TF_demo1_keras.ipynb](https://github.com/google/earthengine-api/blob/master/python/examples/ipynb/TF_demo1_keras.ipynb))*

# Steps followed: 

1.   Exporting training/testing data from Earth Engine in TFRecord format - Four flood events in US.
2.   Preparing the data for use in a TensorFlow model.
2.   Training and validating a Sequential model in TensorFlow.
3.   Making predictions on image data exported from Earth Engine in TFRecord format.
4.   Ingesting classified image data to Earth Engine in TFRecord format.


## Authenticate to Earth Engine



In [ ]:
from google.colab import auth
auth.authenticate_user()

import ee
ee.Authenticate()
ee.Initialize()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Test the TensorFlow installation

Import the TensorFlow library and check the version.

In [ ]:
import tensorflow as tf
print(tf.__version__)
import folium
from pprint import pprint

print(folium.__version__)

# Define variables

This set of global variables will be used throughout.  Need a Cloud Storage bucket to which files will be written. 

In [ ]:
LABEL_DATA = ee.FeatureCollection("users/shahryarahmad/labeldata_floodmap") ## label data prepared from GEE, exported as an asset
print(LABEL_DATA.getInfo())

In [ ]:
#  Earth Engine username.  This is used to import a classified image
# into your Earth Engine assets folder.
USER_NAME = 'shahryarahmad'

# Cloud Storage bucket into which training, testing and prediction 
# datasets will be written.
OUTPUT_BUCKET = 'water-classification-sensitivity'

# Use Sentinel-1 SAR for preliminary classification, conservative threshold of -24dB used
THRESHOLD = -24

s1 = ee.ImageCollection("COPERNICUS/S1_GRD")
srtm = ee.Image("USGS/SRTMGL1_003");

# Use these bands for prediction.
BANDS = ['VV-flood','VH-flood','VV-noflood','VH-noflood','elevation','slope']

# The labels, consecutive integer indices starting from zero, are stored in
# this property, set on each point.
LABEL = 'landuse'
# Number of label values, i.e. number of classes in the classification.
N_CLASSES = 3

# These names are used to specify properties in the export of
# training/testing data and to define the mapping between names and data
# when reading into TensorFlow datasets.
FEATURE_NAMES = list(BANDS)
FEATURE_NAMES.append(LABEL)

# File names for the training and testing datasets.  These TFRecord files
# will be exported from Earth Engine into the Cloud Storage bucket.
TRAIN_FILE_PREFIX = 'Training_water_all' 
TEST_FILE_PREFIX = 'Testing_water_all' 
file_extension = '.tfrecord.gz'
TRAIN_FILE_PATH = 'gs://' + OUTPUT_BUCKET + '/' + TRAIN_FILE_PREFIX + file_extension
TEST_FILE_PATH = 'gs://' + OUTPUT_BUCKET + '/' + TEST_FILE_PREFIX + file_extension


# Get Training and Testing data from Earth Engine


In [ ]:
#//*******************  SENTINEL SAR 1 PROCESSING  ***********************************************************

# Threshold for look angle used to remove erroneous data at far edges of images
angle_threshold_1 = ee.Number(45.4);
angle_threshold_2 = ee.Number(31.66)

# Define focal median function
def focal_median(img):
  fm = img.focal_max(30, 'circle', 'meters')
  # print(fm.getInfo())
  fm = fm.rename(["VV_Smooth", "VH_Smooth"])
  return img.addBands(fm)


def smoothing(img):
  boxcar = ee.Kernel.circle(radius = 1, units = 'pixels', magnitude = 1);

  # Closing operation
  smooth = img.focal_max(kernel = boxcar, iterations = 1).focal_min(kernel = boxcar, iterations = 1)
  # Smooth the image by convolving with the boxcar kernel.
  # smooth = img.convolve(boxcar);
  smooth = smooth.rename("Smooth")
  return img.addBands(smooth)

# Define masking function for removing erroneous pixels
def mask_by_angle(img):
  angle = img.select('angle');
  vv = img.select('VH');
  mask1 = angle.lt(angle_threshold_1);
  mask2 = angle.gt(angle_threshold_2);
  vv = vv.updateMask(mask1);
  return vv.updateMask(mask2);

def mask_by_angle_VV(img):
  angle = img.select('angle');
  vv = img.select('VV');
  mask1 = angle.lt(angle_threshold_1);
  mask2 = angle.gt(angle_threshold_2);
  vv = vv.updateMask(mask1);
  return vv.updateMask(mask2);

### Prepare SAR imagery for training over multiple regions

houston =  ee.Geometry.Point([-95.76422509388891, 29.46698384300668])
florence = ee.Geometry.Point([-78.30901300007325, 34.642647795510356])
missouri = ee.Geometry.Point([-97.925274609375, 40.572107759969555])
arkansas = ee.Geometry.Point([-93.07378231277586, 35.20230235842815])

# eg 4. arkansas
date_range_ar = ee.DateRange('2019-01-01','2019-01-20');
date_range2_ar = ee.DateRange('2019-05-25','2019-05-30');

# eg 3. missouri
date_range_mi = ee.DateRange('2019-08-01','2019-08-20');
date_range2_mi = ee.DateRange('2019-03-13','2019-03-25');


# eg 2. florence
date_range_fl = ee.DateRange('2018-08-07','2018-08-20');
date_range2_fl = ee.DateRange('2018-09-17','2018-09-29');


# eg 1. harvey
date_range_ho = ee.DateRange('2017-06-07','2017-06-09');
date_range2_ho = ee.DateRange('2017-08-25','2017-08-30');


#non-flooded
s1nf_houston = s1\
  .filterDate(date_range_ho)\
  .filterBounds(houston)\
  .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))\
  .filter(ee.Filter.eq('instrumentMode', 'IW'))

s1nf_florence = s1\
  .filterDate(date_range_fl)\
  .filterBounds(florence)\
  .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))\
  .filter(ee.Filter.eq('instrumentMode', 'IW'))

s1nf_arkansas = s1\
  .filterDate(date_range_ar)\
  .filterBounds(arkansas)\
  .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))\
  .filter(ee.Filter.eq('instrumentMode', 'IW'))

s1nf_missouri = s1\
  .filterDate(date_range_mi)\
  .filterBounds(missouri)\
  .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))\
  .filter(ee.Filter.eq('instrumentMode', 'IW'))

s1_noflood = ee.ImageCollection.fromImages([s1nf_houston.first(), s1nf_florence.first(),
                                  s1nf_missouri.first(), s1nf_arkansas.first()]).mosaic()

vh_noflood = mask_by_angle(s1_noflood)
vv_noflood = mask_by_angle_VV(s1_noflood)
vv_noflood = vv_noflood.addBands(vh_noflood.select('VH'))
vv_noflood = focal_median(vv_noflood)  # has 4 bands: VV, VH, VV_Smooth, VH_Smooth
vv_noflood = vv_noflood.select(["VV_Smooth", "VH_Smooth"]).rename(['VV-noflood','VH-noflood'])
 

## Flooded image

s1f_houston = s1\
  .filterDate(date_range2_ho)\
  .filterBounds(houston)\
  .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))\
  .filter(ee.Filter.eq('instrumentMode', 'IW'))

s1f_florence = s1\
  .filterDate(date_range2_fl)\
  .filterBounds(florence)\
  .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))\
  .filter(ee.Filter.eq('instrumentMode', 'IW'))

s1f_arkansas = s1\
  .filterDate(date_range2_ar)\
  .filterBounds(arkansas)\
  .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))\
  .filter(ee.Filter.eq('instrumentMode', 'IW'))

s1f_missouri = s1\
  .filterDate(date_range2_mi)\
  .filterBounds(missouri)\
  .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))\
  .filter(ee.Filter.eq('instrumentMode', 'IW'))

s1_flood = ee.ImageCollection.fromImages([s1f_houston.first(), s1f_florence.first(), s1f_missouri.first(), s1f_arkansas.first()]).mosaic()

vh_flood = mask_by_angle(s1_flood)
vv_flood = mask_by_angle_VV(s1_flood)
vv_flood = vv_flood.addBands(vh_flood.select('VH'))
vv_flood = focal_median(vv_flood)  # has 4 bands: VV, VH, VV_Smooth, VH_Smooth
vv_flood = vv_flood.select(["VV_Smooth", "VH_Smooth"]).rename(['VV-flood','VH-flood'])
 
def radians(img): 
  return img.toFloat().multiply(3.14159).divide(180)

		
## IMAGE TO TRAIN
import math
terrain = ee.Algorithms.Terrain(srtm);
slope = radians(terrain.select('slope'))
aspect = radians(terrain.select('aspect'))

image = vv_flood.addBands(vv_noflood.select('VV-noflood'))\
                .addBands(vv_noflood.select('VH-noflood')) \
                .addBands(srtm.select('elevation').double()) \
                .addBands(slope.select('slope'))
                
print('final image',image.getInfo())

# Use folium to visualize the imagery.
mapid = image.getMapId({'bands': ['VH-noflood'], 'min': -30, 'max': 0})
map = folium.Map(location=[29.4977, -95.36524])

folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='median composite',
  ).add_to(map)
map.add_child(folium.LayerControl())
map

## Add pixel values of the composite to labeled points



In [ ]:
# Sample the image at the points and add a random column.
sample = image.sampleRegions(
  collection=LABEL_DATA, properties=[LABEL], scale=30).randomColumn()

# Partition the sample approximately 70-30.
training = sample.filter(ee.Filter.lt('random', 0.7))
testing = sample.filter(ee.Filter.gte('random', 0.7))


# Print the first couple points to verify.
pprint({'training': training.first().getInfo()})
pprint({'testing': testing.first().getInfo()})

## Export the training and testing data



In [ ]:
# Make sure you can see the output bucket.  You must have write access.
print('Found Cloud Storage bucket.' if tf.io.gfile.exists('gs://' + OUTPUT_BUCKET) 
    else 'Can not find output Cloud Storage bucket.')

In [ ]:
# Create the tasks.
training_task = ee.batch.Export.table.toCloudStorage(
  collection=training,
  description='Training Export',
  fileNamePrefix=TRAIN_FILE_PREFIX,
  bucket=OUTPUT_BUCKET,
  fileFormat='TFRecord',
  selectors=FEATURE_NAMES)

testing_task = ee.batch.Export.table.toCloudStorage(
  collection=testing,
  description='Testing Export',
  fileNamePrefix=TEST_FILE_PREFIX,
  bucket=OUTPUT_BUCKET,
  fileFormat='TFRecord',
  selectors=FEATURE_NAMES)

In [ ]:
# Start the tasks.
training_task.start()
testing_task.start()

### Monitor task progress


In [ ]:
# Print all tasks.
pprint(ee.batch.Task.list())

### Check existence of the exported files



In [ ]:
print('Found training file.' if tf.io.gfile.exists(TRAIN_FILE_PATH) 
    else 'No training file found.')
print('Found testing file.' if tf.io.gfile.exists(TEST_FILE_PATH) 
    else 'No testing file found.')

## Export the imagery to classify



In [ ]:
# File name for the prediction (image) dataset.  The trained model will read
# this dataset and make predictions in each pixel.

## Specify events for testing 
kerala =   ee.Geometry.Point([76.19201793332346, 10.432244570388278])
mozam =   ee.Geometry.Point([34.53572525279313, -19.650222084488128])
srilanka =   ee.Geometry.Point([80.19579864227894,6.100260555152524])
nsw = ee.Geometry.Point([147.97393175933806, -33.3861002551131])
ls =  ee.Geometry.Point([106.7087175776598, 14.869266407333143])

# ID of event for exporting 
floodID = 'nebraska'  #
IMAGE_FILE_PREFIX = 'Image_pixel_'  + floodID + '_'

# The output path for the classified image (i.e. predictions) TFRecord file.
OUTPUT_IMAGE_FILE = 'gs://' + OUTPUT_BUCKET + '/Classified_pixel_' + floodID + '.TFRecord'

# The name of the Earth Engine asset to be created by importing
# the classified image from the TFRecord file in Cloud Storage.
OUTPUT_ASSET_ID = 'users/' + USER_NAME + '/Classified_pixel_' + floodID +'_withdem'


# Export imagery in this region.
## harvey
# EXTENT_TO_CLASSIFY = ee.Geometry.Polygon(
#         [[[-96.47401392967757, 29.681041119196202],
#           [-96.47401392967757, 29.164301460271606],
#           [-95.02381861717757, 29.164301460271606],
#           [-95.02381861717757, 29.681041119196202]]]);    

## eg 2. florence
# EXTENT_TO_CLASSIFY = ee.Geometry.Polygon(
#         [[[-78.67341578514632, 34.712405817452094],
#           [-78.67341578514632, 34.134681521103936],
#           [-77.7986294081932, 34.134681521103936],
#           [-77.7986294081932, 34.712405817452094]]]);

# eg 3. extent_missouri
# EXTENT_TO_CLASSIFY = ee.Geometry.Polygon(
#         [[[-98.7765771200875, 41.479444734972475],
#           [-98.7765771200875, 40.965048314605184],
#           [-97.29616940524375, 40.965048314605184],
#           [-97.29616940524375, 41.479444734972475]]]);

## eg 4. extent_arkansas
# EXTENT_TO_CLASSIFY = ee.Geometry.Polygon(
#         [[[-94.18336451561507, 35.52342267961415],
#           [-94.18336451561507, 35.04812504434383],
#           [-92.52168238670882, 35.04812504434383],
#           [-92.52168238670882, 35.52342267961415]]]);

# ## eg 5. extent_kerala
# EXTENT_TO_CLASSIFY = ee.Geometry.Polygon(
#         [[[76.16033503714692, 10.507022269292058],
#           [76.16033503714692, 10.16792189672564],
#           [76.39860102835786, 10.16792189672564],
#           [76.39860102835786, 10.507022269292058]]])

## eg 5. extent_keralabig
# EXTENT_TO_CLASSIFY =  ee.Geometry.Polygon(
#         [[[76.12188288870942, 10.507022269292058],
#           [76.12188288870942, 10.019197933519012],
#           [76.40684077445161, 10.019197933519012],
#           [76.40684077445161, 10.507022269292058]]])

## eg 6. extent_mozam
# EXTENT_TO_CLASSIFY = ee.Geometry.Polygon(
#         [[[34.20765164882542, -19.55762825258411],
#           [34.20765164882542, -20.23169905138531],
#           [34.74872830898167, -20.23169905138531],
#           [34.74872830898167, -19.55762825258411]]])


## eg 7. extent_sl
# EXTENT_TO_CLASSIFY = ee.Geometry.Polygon(
#         [[[80.15206589484337, 6.1372608569348674],
#           [80.15206589484337, 6.047135541681527],
#           [80.26810898566369, 6.047135541681527],
#           [80.26810898566369, 6.1372608569348674]]])

## eg . extent_ns
# EXTENT_TO_CLASSIFY = ee.Geometry.Polygon(
#         [[[147.74177168252993, -33.27416512270075],
#           [147.74177168252993, -33.575837125375116],
#           [148.24987862947512, -33.575837125375116],
#           [148.24987862947512, -33.27416512270075]]])


## eg . extent_ur
# EXTENT_TO_CLASSIFY = ee.Geometry.Polygon(
#         [[[-58.22185802449711, -31.253794836847117],
#           [-58.22185802449711, -31.607886792184388],
#           [-57.80423541905268, -31.607886792184388],
#           [-57.80423541905268, -31.253794836847117]]])
# ur = ee.Geometry.Point([-58.024852474945625, -31.43463428309628])

# extent_ls 
# EXTENT_TO_CLASSIFY=  ee.Geometry.Polygon(
#         [[[106.36773892289094, 15.361705008559696],
#           [106.36773892289094, 14.543123762883898],
#           [107.12038823014191, 14.543123762883898],
#           [107.12038823014191, 15.361705008559696]]]);

sn = ee.Geometry.Point([-97.205985, 42.791281])
# extent_sn
EXTENT_TO_CLASSIFY =    ee.Geometry.Polygon(
        [[[-97.56520653959635, 43.10070218967456],
          [-97.56520653959635, 42.64280371531401],
          [-96.47685731645944, 42.64280371531401],
          [-96.47685731645944, 43.10070218967456]]]);

# eg 12. nebraska
date_range_sn = ee.DateRange('2018-11-10','2018-11-20');
date_range2_sn = ee.DateRange('2019-03-16','2019-03-20');


#eg 12. ls
# date_range_ls = ee.DateRange('2017-05-18','2017-05-20');
# date_range2_ls = ee.DateRange('2018-07-24','2018-07-26');



#eg 9. ns
# date_range_ns = ee.DateRange('2016-08-01','2016-08-15');
# date_range2_ns = ee.DateRange('2016-09-27','2016-09-28');

# eg 7. sl
# date_range_sl = ee.DateRange('2017-01-10','2017-01-25');
# date_range2_sl = ee.DateRange('2017-05-28','2017-05-31');

# # eg 6. mozam
# date_range_mz = ee.DateRange('2019-03-10','2019-03-15');
# date_range2_mz = ee.DateRange('2019-03-18','2019-03-25');


# eg 5. kerala
# date_range_kr = ee.DateRange('2018-03-01','2018-03-20');
# date_range2_kr = ee.DateRange('2019-08-05','2019-08-12');

# eg 4. arkansas
# date_range = ee.DateRange('2019-01-01','2019-01-20');
# date_range2 = ee.DateRange('2019-05-25','2019-05-30');

# eg 3. missouri
# var date_range = ee.DateRange('2019-08-01','2019-08-20');
# var date_range2 = ee.DateRange('2019-03-13','2019-03-25');

# eg 2. florence
# date_range = ee.DateRange('2018-08-07','2018-08-20');
# date_range2 = ee.DateRange('2018-09-17','2018-09-29');


S1 = s1\
		  .filterDate(date_range_sn)\
		  .filterBounds(sn)\
		  .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))\
		  .filter(ee.Filter.eq('instrumentMode', 'IW')).first()

vh_noflood = mask_by_angle(S1)
vv_noflood = mask_by_angle_VV(S1)
vv_noflood = vv_noflood.addBands(vh_noflood.select('VH'))
vv_noflood = focal_median(vv_noflood)  # has 4 bands: VV, VH, VV_Smooth, VH_Smooth
vv_noflood = vv_noflood.select(["VV_Smooth", "VH_Smooth"]).rename(['VV-noflood','VH-noflood'])
 

S1 = s1\
		  .filterDate(date_range2_sn)\
		  .filterBounds(sn)\
		  .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))\
		  .filter(ee.Filter.eq('instrumentMode', 'IW')).first()

vh_flood = mask_by_angle(S1)
vv_flood = mask_by_angle_VV(S1)
vv_flood = vv_flood.addBands(vh_flood.select('VH'))
vv_flood = focal_median(vv_flood)  # has 4 bands: VV, VH, VV_Smooth, VH_Smooth
vv_flood = vv_flood.select(["VV_Smooth", "VH_Smooth"]).rename(['VV-flood','VH-flood']) 

## IMAGE TO CLASSIFY DURING TESTING
import math
terrain = ee.Algorithms.Terrain(srtm);
slope = radians(terrain.select('slope'))
aspect = radians(terrain.select('aspect'))

image_to_classify = vv_flood.addBands(vv_noflood.select('VV-noflood')) \
                .addBands(vv_noflood.select('VH-noflood')) \
                .addBands(srtm.select('elevation').double()) \
                .addBands(slope.select('slope'))

print(image_to_classify.getInfo())
# Use folium to visualize the imagery.
mapid = image_to_classify.getMapId({'bands': ['VH-flood'], 'min': -30, 'max': 0})
map = folium.Map(location=[42.791281,-97.205985])

folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='median composite',
  ).add_to(map)
map.add_child(folium.LayerControl())
map

In [ ]:
# Specify patch and file dimensions.
image_export_options = {
  'patchDimensions': [256, 256],
  'maxFileSize': 104857600,
  'compressed': True
}

# Setup the task.

image_task = ee.batch.Export.image.toCloudStorage(
  image=image_to_classify,
  description='Image Export',
  fileNamePrefix=IMAGE_FILE_PREFIX,
  bucket=OUTPUT_BUCKET,
  scale=30,
  fileFormat='TFRecord',
  region=EXTENT_TO_CLASSIFY.toGeoJSON()['coordinates'],
  formatOptions=image_export_options,
)

In [ ]:
# Start the task.
image_task.start()

### Monitor task progress

In [ ]:
import time

while image_task.active():
  print('Polling for task (id: {}).'.format(image_task.id))
  print(image_task.status())
  time.sleep(30)
print('Done with image export.')

# Data preparation and pre-processing

Read data from the TFRecord file into a `tf.data.Dataset`.  Pre-process the dataset to get it into a suitable format for input to the model.

## Read into a `tf.data.Dataset`




In [ ]:
# Create a dataset from the TFRecord file in Cloud Storage.
train_dataset = tf.data.TFRecordDataset(TRAIN_FILE_PATH, compression_type='GZIP')
# Print the first record to check.
print(iter(train_dataset).next())

## Define the structure of your data



In [ ]:
# List of fixed-length features, all of which are float32.
columns = [
  tf.io.FixedLenFeature(shape=[1], dtype=tf.float32) for k in FEATURE_NAMES
]

# Dictionary with names as keys, features as values.
features_dict = dict(zip(FEATURE_NAMES, columns))

pprint(features_dict)

## Parse the dataset



In [ ]:
def parse_tfrecord(example_proto):
  """The parsing function.

  Read a serialized example into the structure defined by featuresDict.

  Args:
    example_proto: a serialized Example.

  Returns:
    A tuple of the predictors dictionary and the label, cast to an `int32`.
  """
  parsed_features = tf.io.parse_single_example(example_proto, features_dict)
  labels = parsed_features.pop(LABEL)
  return parsed_features, tf.cast(labels, tf.int32)

# Map the function over the dataset.
parsed_dataset = train_dataset.map(parse_tfrecord, num_parallel_calls=5)

def to_tuple(inputs, label):
  return (tf.transpose(list(inputs.values())),
          tf.one_hot(indices=label, depth=N_CLASSES))

# Print the first parsed record to check.
pprint(iter(parsed_dataset).next())

# NN Model setup

The workflow for classification in TensorFlow is:

1.  Create the model.
2.  Train the model (i.e. `fit()`).
3.  Use the trained model for inference (i.e. `predict()`).


## Create the Keras model



In [ ]:
from tensorflow import keras

input_dataset = parsed_dataset 

# Keras requires inputs as a tuple.  Note that the inputs must be in the
# right shape.  Also note that to use the categorical_crossentropy loss,
# the label needs to be turned into a one-hot vector.
def to_tuple(inputs, label):
  return (tf.transpose(list(inputs.values())),
          tf.one_hot(indices=label, depth=N_CLASSES))

# Map the to_tuple function, shuffle and batch.
input_dataset = input_dataset.map(to_tuple).batch(8)

# Define the layers in the model.
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(256, activation=tf.nn.relu,kernel_regularizer=tf.keras.regularizers.l2(l = 0.001)),
  tf.keras.layers.Dropout(0.4),
  tf.keras.layers.Dense(128, activation=tf.nn.relu,kernel_regularizer=tf.keras.regularizers.l2(l = 0.001)),
  tf.keras.layers.Dropout(0.4),
  tf.keras.layers.Dense(64, activation=tf.nn.relu,kernel_regularizer=tf.keras.regularizers.l2(l = 0.001)),
  tf.keras.layers.Dropout(0.4),
  tf.keras.layers.Dense(32, activation=tf.nn.relu),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(16, activation=tf.nn.relu),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(N_CLASSES, activation=tf.nn.softmax)
])

# Compile the model with the specified loss function.
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['categorical_accuracy'])

# Fit the model to the training data.
model.fit(x=input_dataset, epochs=1000)

## Check model accuracy on the test set



In [ ]:
test_dataset = (
  tf.data.TFRecordDataset(TEST_FILE_PATH, compression_type='GZIP')
    .map(parse_tfrecord, num_parallel_calls=5)
    .map(to_tuple)
    .batch(1))

model.evaluate(test_dataset)


# Testing - Use the trained model to classify an image from Earth Engine



In [ ]:
test_dataset = (
  tf.data.TFRecordDataset(TEST_FILE_PATH, compression_type='GZIP')
    .map(parse_tfrecord, num_parallel_calls=5)
    .map(to_tuple)
    .batch(1))

## Load pre-saved model

loaded_model = tf.keras.models.load_model('/content/drive/My Drive/flood-mapping-GEEexports/model_withdem_0p95')
model = loaded_model
model.evaluate(test_dataset)

## Find the image files and JSON mixer file in Cloud Storage



In [ ]:
# Get a list of all the files in the output bucket.
files_list = !gsutil ls 'gs://'{OUTPUT_BUCKET}
# Get only the files generated by the image export.
exported_files_list = [s for s in files_list if IMAGE_FILE_PREFIX in s]

# Get the list of image files and the JSON mixer file.
image_files_list = []
json_file = None
for f in exported_files_list:
  if f.endswith('.tfrecord.gz'):
    image_files_list.append(f)
  elif f.endswith('.json'):
    json_file = f

# Make sure the files are in the right order.
image_files_list.sort()

pprint(image_files_list)
print(json_file)

## Read the JSON mixer file



In [ ]:
import json

# Load the contents of the mixer file to a JSON object.
json_text = !gsutil cat {json_file}
# Get a single string w/ newlines from the IPython.utils.text.SList
mixer = json.loads(json_text.nlstr)
pprint(mixer)

print(json_file)

## Read the image files into a dataset



In [ ]:
# Get relevant info from the JSON mixer file.
patch_width = mixer['patchDimensions'][0]
patch_height = mixer['patchDimensions'][1]
patches = mixer['totalPatches']
patch_dimensions_flat = [patch_width * patch_height, 1]

# Note that the tensors are in the shape of a patch, one patch for each band.
image_columns = [
  tf.io.FixedLenFeature(shape=patch_dimensions_flat, dtype=tf.float32) 
    for k in BANDS
]

# Parsing dictionary.
image_features_dict = dict(zip(BANDS, image_columns))

# Note that you can make one dataset from many files by specifying a list.
image_dataset = tf.data.TFRecordDataset(image_files_list, compression_type='GZIP')

# Parsing function.
def parse_image(example_proto):
  return tf.io.parse_single_example(example_proto, image_features_dict)

# Parse the data into tensors, one long tensor per patch.
image_dataset = image_dataset.map(parse_image, num_parallel_calls=5)

# Break our long tensors into many little ones.
image_dataset = image_dataset.flat_map(
  lambda features: tf.data.Dataset.from_tensor_slices(features)
)

# Turn the dictionary in each record into a tuple without a label.
image_dataset = image_dataset.map(
  lambda data_dict: (tf.transpose(list(data_dict.values())), )
)

# Turn each patch into a batch.
image_dataset = image_dataset.batch(patch_width * patch_height)

## Generate predictions for the image pixels

To get predictions in each pixel, run the image dataset through the trained model using `model.predict()`.  Print the first prediction to see that the output is a list of the three class probabilities for each pixel.  Running all predictions might take a while.

In [ ]:
# Run prediction in batches, with as many steps as there are patches.
predictions = model.predict(image_dataset, steps=patches, verbose=1)

# Note that the predictions come as a numpy array.  Check the first one.
print(predictions[0])

## Write the predictions to a TFRecord file



In [ ]:
print('Writing to file ' + OUTPUT_IMAGE_FILE)

In [ ]:
# Instantiate the writer.
writer = tf.io.TFRecordWriter(OUTPUT_IMAGE_FILE)

# Every patch-worth of predictions we'll dump an example into the output
# file with a single feature that holds our predictions. Since our predictions
# are already in the order of the exported data, the patches we create here
# will also be in the right order.
patch = [[], [], [], []]
cur_patch = 1
for prediction in predictions:
  patch[0].append(tf.argmax(prediction, 1))
  patch[1].append(prediction[0][0])
  patch[2].append(prediction[0][1])
  patch[3].append(prediction[0][2])
  # Once we've seen a patches-worth of class_ids...
  if (len(patch[0]) == patch_width * patch_height):
    print('Done with patch ' + str(cur_patch) + ' of ' + str(patches) + '...')
    # Create an example
    example = tf.train.Example(
      features=tf.train.Features(
        feature={
          'prediction': tf.train.Feature(
              int64_list=tf.train.Int64List(
                  value=patch[0])),
          'nowater': tf.train.Feature(
              float_list=tf.train.FloatList(
                  value=patch[1])),
          'permwater': tf.train.Feature(
              float_list=tf.train.FloatList(
                  value=patch[2])),
          'floodwater': tf.train.Feature(
              float_list=tf.train.FloatList(
                  value=patch[3])),
        }
      )
    )
    # Write the example to the file and clear our patch array so it's ready for
    # another batch of class ids
    writer.write(example.SerializeToString())
    patch = [[], [], [], []]
    cur_patch += 1

writer.close()

# Upload the classifications to an Earth Engine asset

## Verify the existence of the predictions file



In [ ]:
!gsutil ls -l {OUTPUT_IMAGE_FILE}

## Upload the classified image to Earth Engine



In [ ]:
print('Uploading to ' + OUTPUT_ASSET_ID)

In [ ]:
# Start the upload.
# json_file = 'gs://water-classification-sensitivity/Image_pixel_laos_mixer.json'

print(json_file)
!earthengine upload image --asset_id={OUTPUT_ASSET_ID} --pyramiding_policy=mode {OUTPUT_IMAGE_FILE} {json_file}

## Check the status of the asset ingestion



In [ ]:
ee.batch.Task.list()


## View the ingested asset in GEE

